In [1]:
import ccloud_lib
from faker import Faker
from uuid import uuid4

from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import StringSerializer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer

In [2]:
TOPIC = "fake-users"

In [3]:
conf = ccloud_lib.read_ccloud_config("python.config")

In [72]:
# Create topic if needed
ccloud_lib.create_topic(conf, TOPIC)

%4|1658046050.142|CONFWARN|rdkafka#producer-14| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance


Topic fake-users created


In [4]:
class User(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [5]:
def user_to_dict(user, ctx):
    return dict(name=user.name,
                age=user.age)

In [6]:
user_schema = """
{
    "namespace": "confluent.io.examples.serialization.avro",
    "name": "User",
    "type": "record",
    "fields": [
        {
            "name": "name",
            "type": "string"
        },
        {
            "name": "age",
            "type": "int"
        }
    ]
}
"""

In [7]:
schema_registry_conf = {
    'url': conf['schema.registry.url'],
    'basic.auth.user.info': conf['basic.auth.user.info']}

schema_registry_client = SchemaRegistryClient(schema_registry_conf)

avro_serializer = AvroSerializer(schema_registry_client,
                                 user_schema,
                                 user_to_dict)

In [8]:
# Create Producer instance
producer_conf = ccloud_lib.pop_schema_registry_params_from_config(conf)
producer_conf['key.serializer'] = StringSerializer('utf_8')
producer_conf['value.serializer'] = avro_serializer
producer = SerializingProducer(producer_conf)

%4|1658941908.666|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance


In [9]:
fake=Faker()

In [14]:
for i in range(1000):
    name = fake.name()
    age = fake.random_int(min=18, max=80, step=1)
    user = User(name=name,
                age=age)
    producer.produce(topic=TOPIC,
                     key=str(uuid4()),
                     value=user,
                     on_delivery=ccloud_lib.acked)
    producer.poll(0)

Produced record to topic fake-users partition [0] @ offset 4050
Produced record to topic fake-users partition [0] @ offset 4051
Produced record to topic fake-users partition [0] @ offset 4052
Produced record to topic fake-users partition [0] @ offset 4053
Produced record to topic fake-users partition [0] @ offset 4054
Produced record to topic fake-users partition [0] @ offset 4055
Produced record to topic fake-users partition [0] @ offset 4056
Produced record to topic fake-users partition [0] @ offset 4057
Produced record to topic fake-users partition [0] @ offset 4058
Produced record to topic fake-users partition [0] @ offset 4059
Produced record to topic fake-users partition [0] @ offset 4060
Produced record to topic fake-users partition [0] @ offset 4061
Produced record to topic fake-users partition [0] @ offset 4062
Produced record to topic fake-users partition [0] @ offset 4063
Produced record to topic fake-users partition [0] @ offset 4064
Produced record to topic fake-users part

In [13]:
producer.flush()

Produced record to topic fake-users partition [0] @ offset 3199
Produced record to topic fake-users partition [0] @ offset 3200
Produced record to topic fake-users partition [0] @ offset 3201
Produced record to topic fake-users partition [0] @ offset 3202
Produced record to topic fake-users partition [0] @ offset 3203
Produced record to topic fake-users partition [0] @ offset 3204
Produced record to topic fake-users partition [0] @ offset 3205
Produced record to topic fake-users partition [0] @ offset 3206
Produced record to topic fake-users partition [0] @ offset 3207
Produced record to topic fake-users partition [0] @ offset 3208
Produced record to topic fake-users partition [0] @ offset 3209
Produced record to topic fake-users partition [0] @ offset 3210
Produced record to topic fake-users partition [0] @ offset 3211
Produced record to topic fake-users partition [0] @ offset 3212
Produced record to topic fake-users partition [0] @ offset 3213
Produced record to topic fake-users part

0